# 동의어사전 - 한글을 영어로 친 경우 찾기

2020-11-16  
&nbsp;  
동의어 사전에서 *사과 = tkrhk*라고 등록된 경우를 찾는다.  
= 한글 키워드를 키보드대로 영문 입력된 것

In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings(action='ignore')

파일불러오기

In [2]:
sd = pd.read_excel('SynonymDictionary.xlsx')
sd = sd[['표제어', '동의어']]

### 표제어가 한글

표제어가 한글일 것 → 표제어가 영어로만 이루어진 것 제외  
&nbsp;  
이유 : 영어를 영어로 변환해봤자 무의미하고 불필요한 것들까지 인식하게 됨

In [3]:
# 영문인 것 제외
com = re.compile('^[a-zA-Z0-9_]+$')
sd['표제어_Eng'] = sd['표제어'].apply(lambda x: '' if com.match(x)==None else com.match(x).group())

# 동의어 사전_대상
sd_obj = sd[sd['표제어_Eng']==''].reset_index(drop = True)

### 한글 자음/모음 분리 → 영어로 변환

사과 → ㅅㅏㄱㅘ

In [4]:
from jamo import h2j, j2hcj

sd_obj['표제어_분리'] = sd_obj['표제어'].apply(lambda x: j2hcj(h2j(x)))

한글-영문에 맞추어 변환

In [5]:
keyboard = {'ㅂ':'q', 'ㅈ':'w', 'ㄷ':'e', 'ㄱ':'r', 'ㅅ':'t', 'ㅛ':'y', 'ㅕ':'u', 'ㅑ':'i', 'ㅐ':'o', 'ㅔ':'p',
            'ㅁ':'a', 'ㄴ':'s', 'ㅇ':'d', 'ㄹ':'f', 'ㅎ':'g', 'ㅗ':'h', 'ㅓ':'j', 'ㅏ':'k', 'ㅣ':'l',
            'ㅋ':'z', 'ㅌ':'x', 'ㅊ':'c', 'ㅍ':'v', 'ㅠ':'b', 'ㅜ':'n', 'ㅡ':'m',
           'ㅘ':'hk', 'ㅙ':'ho', 'ㅚ':'hl', 'ㅝ':'nj', 'ㅞ':'np', 'ㅟ':'nl', 'ㅢ':'ml'}

def kkeyeng(kwd):
    return ''.join([keyboard[w] if w in keyboard else w for w in kwd])

In [6]:
sd_obj['표제어_Eng'] = sd_obj['표제어_분리'].apply(lambda x: kkeyeng(x))

영문 소문자로 통일

In [7]:
sd_obj['동의어'] = sd_obj['동의어'].str.lower()
sd_obj['표제어_Eng'] = sd_obj['표제어_Eng'].str.lower()

변환한 영어표제어가 동의어 목록에 있는지 찾기

In [8]:
sd_obj['찾았다'] = ''

for i in range(0, len(sd_obj)):
    if sd_obj['표제어_Eng'][i] in sd_obj['동의어'][i]:
        sd_obj['찾았다'][i] = '얏호!'

### 결과

In [9]:
print('찾은 갯수 :', len(sd_obj[sd_obj['찾았다']=='얏호!']))

찾은 갯수 : 48


In [10]:
sd_obj[sd_obj['찾았다']=='얏호!'].head()

,표제어,동의어,표제어_Eng,표제어_분리,찾았다
131,물티슈,"물휴지,anfxltb",anfxltb,ㅁㅜㄹㅌㅣㅅㅠ,얏호!
132,휴지,"티슈,tissue,gbwl",gbwl,ㅎㅠㅈㅣ,얏호!
187,매트,"mat,aoxm,메트",aoxm,ㅁㅐㅌㅡ,얏호!
216,헤드셋,"해드샛,헤드샛,해드셋,headset,gpemtpt",gpemtpt,ㅎㅔㄷㅡㅅㅔㅅ,얏호!
217,햇반,"haetban,hatban,gotqks",gotqks,ㅎㅐㅅㅂㅏㄴ,얏호!


내보내기

In [11]:
sd_obj[sd_obj['찾았다']=='얏호!'][['표제어', '동의어', '표제어_Eng']].to_excel('영문동의어_영타.xlsx', index = False)